# 拉勾评论分析


In [ ]:
'''
Created on Tue May 24 18:25:41 2016

@author: 王超(xchao)
'''

## Get Data by Crawler 

In [ ]:
#-*—coding:utf8-*-
import requests
import json
import pandas as pd
from pandas import read_json
from pandas.io.json import json_normalize
#handle encoding
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
import time
from functools import wraps

# timer
def fn_timer(function):
  @wraps(function)
  def function_timer(*args, **kwargs):
    t0 = time.time()
    result = function(*args, **kwargs)
    t1 = time.time()
    print ("Total time running %s: %s seconds" %
        (function.func_name, str(t1-t0))
        )
    return result
  return function_timer

# handle async loading
asyncData = {
    'companyId':'147',
    'positionType':'',
    'pageSize':'10',
    'pageNo':'1',
}

@fn_timer
def getDataframe(pageNo,lagouJson,asyncData):
    """
    pageNo : comment page num 1-110
    lagouJson : www
    asyncData : async loading dic
    rtype : pd.Dataframe
    """
    for i in range(1,pageNo+1):
        asyncData['pageNo']= str(i)
        html_post = requests.post(lagouJson, data=asyncData)
        html_post.encoding = 'utf-8'
        print html_post.text
        data = json.loads(html_post.text)
        if i==1:
            dataframe=json_normalize(data['content']['data']['page']['result'])
        else:
            dataframe=dataframe.append(json_normalize(data['content']['data']['page']['result']))
        print('page : ' + str(i))
        print(dataframe.shape)
    return dataframe

if __name__ == "__main__":
    lagouJson = 'http://www.lagou.com/gongsi/searchInterviewExperiences.json'
    pageNo = 110
    data1 = getDataframe(pageNo,lagouJson,asyncData)
    #save the data
    data1.to_csv('data_json.csv')

## Read Data

In [2]:
import pandas as pd
import numpy as np
# plot
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

import nltk
import jieba.analyse

import sys
reload(sys)
sys.setdefaultencoding("utf-8")

#   ============ version =======
print nltk.__version__
print jieba.__version__

In [3]:
# ============ feature select ==========
def featureSelect(df):
    featureColums=['isAnonymous','usefulCount','comprehensiveScore',\
    'content','positionType','createTime','replyCount','tags']
    df = df[featureColums]
    #change time
    df.createTime = df.createTime.str[0:10]
    df.index = df.createTime
    df.index =pd.to_datetime(df.index)  
    #usefulCount +1 = weights
    #df.usefulCount = df.usefulCount + 1
    df['tag_used'] = df.tags.isnull()
    df['tag_used'] = df['tag_used'].map( {True: 0, False: 1} ).astype(int)
    return df

In [18]:
# read data ,preview
file_name = '/Users/xchao/pyDataScienceBase/lagouComment/data_json.csv'
df = pd.read_csv(file_name)
df = featureSelect(df)
df.head()

,isAnonymous,usefulCount,comprehensiveScore,content,positionType,createTime,replyCount,tags,tag_used
createTime,,,,,,,,,
2016-05-24,False,0,4.7,先笔试，四个题(介绍最熟悉的系统架构，HASHMAP细节，算法，数据结构)。面试过程比较纠结...,技术,2016-05-24,0,"[""面试官很nice"",""聊得很开心"",""面试效率高""]",1
2016-05-24,False,0,5.0,电话沟通的，面试官很贴心,市场与销售,2016-05-24,0,"[""面试官很nice""]",1
2016-05-21,True,0,4.0,面试官干练，只是聊的时间很短。自己也过于拘谨了,职能,2016-05-21,0,"[""面试效率高""]",1
2016-05-21,True,0,5.0,面试官人都很好，公司环境也很不错，期待加入！,技术,2016-05-21,0,"[""面试官很nice"",""面试效率高"",""环境高大上""]",1
2016-05-20,True,1,5.0,环境很不错，办公氛围也挺好的，大致的介绍一下职业规划，以前工作内容，可能面试表现不太好，说是...,运营,2016-05-20,1,"[""环境高大上""]",1


In [22]:
df.describe()

,isAnonymous,usefulCount,comprehensiveScore,replyCount,tag_used
count,1100,1100.000000,1100.000000,1100.000000,1100.000000
mean,0.642727,3.998182,4.264273,0.040000,0.897273
std,0.479414,4.652824,0.962503,0.196048,0.303740
min,False,0.000000,1.000000,0.000000,0.000000
25%,0,1.000000,4.000000,0.000000,1.000000
50%,1,3.000000,4.700000,0.000000,1.000000
75%,1,5.000000,5.000000,0.000000,1.000000
max,True,63.000000,5.000000,1.000000,1.000000


In [5]:
# use jieba & nltk , ti-fdf analysis

def freq_output(df,n):
    '''
    func to output the topN tf-idf
    df : filtered df
    n  : top n    
    return: top n list
    '''
    clean_content=[]
    for i in xrange( 0, len(df["content"])):
        clean_content += review_to_wordlist(df["content"][i])
    topN = freq(clean_content,n)
        #clean_content.append(review_to_wordlist(df["content"][i]))
        #clean_content.append(" ".join(review_to_wordlist(df["content"][i])))
    return topN    
    
def freq(clean_content,n):
    #clean_content is list,  n is topN
    freq_dist = nltk.FreqDist(clean_content)
    freq = freq_dist.most_common(n) 
    # return tuple [('a',1)('b',2)...]
    topN = []
    for i in range(0,len(freq)):
        print freq[i][0]
        topN.append(freq[i][0])
    return topN
    
def review_to_wordlist(review): 
    # func to clean train data       
    review_text = BeautifulSoup(review).get_text() # remove HTML
    #print ('review_text : ' +str(len(review_text)))
    # seg_list = jieba.cut(text cut_all=False)
    words = jieba.analyse.extract_tags(review_text)  #split bu jieba 
    return words # return list of words  


In [12]:
df = df[df.comprehensiveScore<3] #filter bad score
topn = freq_output(df,30) #output the top30 tfidf in bad score 

0	面试
1	面试官
2	没有
3	通知
4	感觉
5	公司
6	时间
7	什么
8	电话
9	HR
10	拉勾
11	专业
12	直接
13	自己
14	失望
15	简历
16	就是
17	沟通
18	结果
19	然后
20	还是
21	网站
22	环境
23	分钟
24	合适
25	像是
26	大老远
27	问题
28	打电话
29	建议

In [13]:
# caculate year statics, positionType statics
def yearfilter(df):
     df = df['2015']
     #df=df[df.comprehensiveScore<=3]
     #df=df[df.comprehensiveScore >0]
     month = ['2015-1','2015-2','2015-3','2015-4','2015-5','2015-6','2015-7','2015-8','2015-9',\
              '2015-10','2015-11','2015-12']
     list12= []
     for i in month:
         #list12.append(df[i].content.count())
         #list12.append(df[i].replyCount.sum())
         list12.append(df[i].comprehensiveScore.mean())
     return list12

def posStat(df) :
     df = df['2015']
     pos = ['技术','市场与销售','运营','设计','职能','产品']
     listPos = []
     for i in pos:
         #listPos.append(df[df.positionType==i].comprehensiveScore.mean())
         #listPos.append(df[df.positionType==i].content.count())
         df1=df[df.positionType==i]
         listPos.append(df1[df1.comprehensiveScore<=3].content.count())
     return listPos

In [14]:
list12 = yearfilter(df)
listPos =  posStat(df) 

In [15]:
# generate dic, sentiment analysis

def generateDic():
    dic_guan1_str = '面试官好 面试官很好 面试官人好 面试官人很nice 面试官很耐心 面试官很有耐心 面试官挺耐心 面试官不错 面试官很随和 和面试官沟通的不错 面试官很专业 面试官比较专业 面试官还行 面试官特别随和 感谢面试官 面试官为人谦虚 面试官很温和 面试官很nice 面试官人挺好 面试官挺亲切 面试官人挺亲切 面试官很亲切 面试官人不错 面试官很友好 面试官态度很好 面试官态度挺好 面试官态度不错 面试官很贴心 面试官非常热情 面试官很热情 面试官非常好 面试官很帅 面试官挺帅 面试官美美哒 面试官很美 面试官很漂亮 面试沟通顺畅 面试官也挺亲切 面试官人特别好 面试官特别好 平易近人 面试官比较热情 面试官也很好 面试官也不错 面试官很温柔 面试官很和蔼 面试官很赞 超nice 负责的面试官 面试官很负责 超级nice'
    dic_guan1_list=dic_guan1_str.split(' ')
    dic_guan1_set = set(dic_guan1_list)
    
    dic_guan0_str = '面试官不好 面试官没有做功课 面试官没耐心 面试官不怎么样 面试官不专业 和面试官沟通的不好 面试官不太专业 面试官不行 面试官不随和 面试官垃圾 面试官傲慢 面试官不尊重人 面试官人品有问题 面试官人品不好 面试官不友好 面试官态度不好 面试官不客气 面试官很冷漠 面试官很冷淡 面试官不热情 面试官一般 面试官很菜 什么面试官 面试沟通的不顺畅 面试官水平不够 面试官没水平 面试官特别差 面试官差 面试官特别烂 面试官特别没水平 面试官特别 不专业 面试官态度比较冷 面试官很冷 不平易近人 面试官不热情 面试官也不好 面试官很水 人事很水 面试官丧着个脸 面试官很丧 面试官没什么水平 面试官高冷 态度高冷 态度不好 态度恶劣 态度差 态度特别差 冷冰冰的 面试官不负责'
    dic_guan0_list=dic_guan0_str.split(' ')
    dic_guan0_set = set(dic_guan0_list)
    
    dic_huan1_str = '环境非常好 环境很好 环境好 环境不错 环境还行 环境挺好 环境很不错 环境很棒 环境特别好 环境也不错 环境舒适 环境优雅 环境nice  环境也好 环境超好 喜欢这样的环境'
    dic_huan1_list=dic_huan1_str.split(' ')
    dic_huan1_set = set(dic_huan1_list)
    
    dic_huan0_str = '环境一般 环境像库房 地方比较小 地方挤 挤 不喜欢这个环境'
    dic_huan0_list=dic_huan0_str.split(' ')
    dic_huan0_set = set(dic_huan0_list)
    
    dic_money1_str = '待遇与招聘信息相符 福利待遇杠杠的 工资有竞争力 待遇有竞争力 待遇不错 待遇还行 待遇可以'
    dic_money1_list=dic_money1_str.split(' ')
    dic_money1_set = set(dic_money1_list)
    
    dic_money0_str = '待遇不好 待遇很差 待遇差 待遇一般 待遇与招聘信息不符 待遇不符'
    dic_money0_list=dic_money0_str.split(' ')
    dic_money0_set = set(dic_money0_list)
    
    dic_time1_str = '效率高 很快 注重效率 有效率 效率很高 效率特别高 效率都蛮好的 面试挺快的'
    dic_time1_list=dic_time1_str.split(' ')
    dic_time1_set = set(dic_time1_list)
    
    dic_time0_str = '等了半小时 浪费时间 浪费面试者时间 等了半天 等了很长时间 等了很久 hr不在 HR不在 前台不在 没人接待'
    dic_time0_list=dic_time0_str.split(' ')
    dic_time0_set = set(dic_time0_list)
    return dic_guan1_set,dic_guan0_set,dic_huan1_set,dic_huan0_set,dic_money1_set,dic_money0_set,dic_time1_set,dic_time0_set

def sentiment(df):
    g1,g0,h1,h0,m1,m0,t1,t0=generateDic()
    result = {'g1':0,'g0':0,'h1':0,'h0':0,'m1':0,'m0':0,'t1':0,'t0':0}
    for i in xrange( 0, len(df["content"])):
        for j in g1:
            if j in df["content"][i]:
                result['g1'] +=1
                break
        for j in g0:
            if j in df["content"][i]:
                result['g0'] +=1            
                break
        for j in h1:
            if j in df["content"][i]:
                result['h1'] +=1
                break
        for j in h0:
            if j in df["content"][i]:
                result['h0'] +=1            
                break
        for j in m1:
            if j in df["content"][i]:
                result['m1'] +=1
                break
        for j in m0:
            if j in df["content"][i]:
                result['m0'] +=1            
                break
        for j in t1:
            if j in df["content"][i]:
                result['t1'] +=1
                break
        for j in t0:
            if j in df["content"][i]:
                result['t0'] +=1            
                break  
    return result   

In [23]:
    df = df['2016']
    result = sentiment(df)
    result

{'g0': 14, 'g1': 58, 'h0': 5, 'h1': 363, 'm0': 0, 'm1': 3, 't0': 6, 't1': 14}

## 问题记录：
* 样本量太小，分析可能存在偏差
* tf-idf处理后，合并同义词不好处理
* 目前情感词典为demo，日后根据工作需要，对招聘多维度信息建立词典，有助于标签自动生成

## toDoList
* 情感分析-->意见挖掘
    * 通过综合得分确定积极消极,
    * 确定主观或者客观，
    * 升级到意见挖掘，在情感分析的基础上，找到面试相关的各项属性，再分析对应属性的情感倾向与强度。
* 尝试词向量
* 针对招聘领域，完善词典